In [3]:
import garminconnect
import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
email = os.getenv("email")
password = os.getenv("garminpassword")\

garmin = garminconnect.Garmin(email, password)
garmin.login()
garmin.display_name

'5f3dc0bc-4916-4755-894b-cca99162502b'

In [5]:
import json
date = '2025-06-12'
def getdata(date):
    return garmin.get_sleep_data(date)

data = getdata(date)
print(data)

{'dailySleepDTO': {'id': 1749687960000, 'userProfilePK': 80897268, 'calendarDate': '2025-06-12', 'sleepTimeSeconds': 24660, 'napTimeSeconds': 0, 'sleepWindowConfirmed': True, 'sleepWindowConfirmationType': 'enhanced_confirmed_final', 'sleepStartTimestampGMT': 1749687960000, 'sleepEndTimestampGMT': 1749713640000, 'sleepStartTimestampLocal': 1749691560000, 'sleepEndTimestampLocal': 1749717240000, 'autoSleepStartTimestampGMT': None, 'autoSleepEndTimestampGMT': None, 'sleepQualityTypePK': None, 'sleepResultTypePK': None, 'unmeasurableSleepSeconds': 0, 'deepSleepSeconds': 6960, 'lightSleepSeconds': 14160, 'remSleepSeconds': 3540, 'awakeSleepSeconds': 1020, 'deviceRemCapable': True, 'retro': False, 'sleepFromDevice': True, 'averageSpO2Value': 95.0, 'lowestSpO2Value': 87, 'highestSpO2Value': 100, 'averageSpO2HRSleep': 57.0, 'averageRespirationValue': 15.0, 'lowestRespirationValue': 10.0, 'highestRespirationValue': 20.0, 'awakeCount': 1, 'avgSleepStress': 19.0, 'ageGroup': 'ADULT', 'sleepScore

In [18]:
def getsleep(date):
    data = getdata(date)

    # Safeguard: if no data or missing expected structure
    if not data or not isinstance(data, dict) or 'dailySleepDTO' not in data:
        return pd.DataFrame()

    dailysleep_dto = data.get('dailySleepDTO', {})

    sleepdata = {
        'date': dailysleep_dto.get('calendarDate'),
        'startTime': dailysleep_dto.get('sleepStartTimestampLocal'),
        'endTime': dailysleep_dto.get('sleepEndTimestampLocal'),
        'sleepDuration': dailysleep_dto.get('sleepTimeSeconds'),
        'deepSleepDuration': dailysleep_dto.get('deepSleepSeconds'),
        'lightSleepDuration': dailysleep_dto.get('lightSleepSeconds'),
        'remSleepDuration': dailysleep_dto.get('remSleepSeconds'),
        'spo2Avg': dailysleep_dto.get('averageSpO2Value'),
        'spo2Low': dailysleep_dto.get('lowestSpO2Value'),
        'spo2High': dailysleep_dto.get('highestSpO2Value'),
        'respirationAvg': dailysleep_dto.get('averageRespirationValue'),
        'respirationLow': dailysleep_dto.get('lowestRespirationValue'),
        'respirationHigh': dailysleep_dto.get('highestRespirationValue'),
        'awakeCounter': dailysleep_dto.get('awakeCount'),
        'avgOvernightHrv': data.get('avgOvernightHrv'),
        'restingHeartRate': data.get('restingHeartRate'),
        'restlessmoment': data.get('restlessMomentsCount')
    }

    df_sleepdata = pd.DataFrame([sleepdata])

    # Convert time columns if present, otherwise NaT
    for col in ['startTime', 'endTime']:
        if pd.notnull(df_sleepdata.at[0, col]):
            df_sleepdata[col] = pd.to_datetime(df_sleepdata[col], unit='ms')
        else:
            df_sleepdata[col] = pd.NaT

    return df_sleepdata

In [7]:
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
dbname = os.getenv("dbname")

DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(DATABASE_URL)

try:
    with engine.connect() as conn:
        result = result = conn.execute(text("SELECT version();"))
        for row in result:
            print("Connected to:", row[0])
except Exception as e:
    print(" Connection failed:", e)

Connected to: PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


In [20]:
def loopbackwards(start_date_str, engine, max_misses=7):
    current_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d").date()
    misses = 0
    while misses < max_misses:
            print(f"Processing date: {current_date}")
            df = getsleep(current_date)
            
            if df.empty:
                print(f"No sleep data for {current_date}. Skipping.")
                misses += 1
            else:
                df.to_sql('simplesleepdata', con=engine, if_exists="append", index=False)
                print(f"Inserted data for {current_date}")
                

            current_date -= datetime.timedelta(days=1)
                
    print("Stopped after too many missing days.")


today = datetime.date.today().strftime("%Y-%m-%d")

loopbackwards(today, engine)


Processing date: 2025-06-26
Inserted data for 2025-06-26
Processing date: 2025-06-25
Inserted data for 2025-06-25
Processing date: 2025-06-24
Inserted data for 2025-06-24
Processing date: 2025-06-23
Inserted data for 2025-06-23
Processing date: 2025-06-22
Inserted data for 2025-06-22
Processing date: 2025-06-21
Inserted data for 2025-06-21
Processing date: 2025-06-20
Inserted data for 2025-06-20
Processing date: 2025-06-19
Inserted data for 2025-06-19
Processing date: 2025-06-18
Inserted data for 2025-06-18
Processing date: 2025-06-17
Inserted data for 2025-06-17
Processing date: 2025-06-16
Inserted data for 2025-06-16
Processing date: 2025-06-15
Inserted data for 2025-06-15
Processing date: 2025-06-14
Inserted data for 2025-06-14
Processing date: 2025-06-13
Inserted data for 2025-06-13
Processing date: 2025-06-12
Inserted data for 2025-06-12
Processing date: 2025-06-11
Inserted data for 2025-06-11
Processing date: 2025-06-10
Inserted data for 2025-06-10
Processing date: 2025-06-09
Ins

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

**Create Recovery Score**

In [38]:
query = text('SELECT date, "avgOvernightHrv", "restingHeartRate", "sleepDuration" FROM simplesleepdata')
df = pd.read_sql(query, con=engine)

df["date"] = pd.to_datetime(df["date"])

df["sleep_hours"] = df["sleepDuration"] / 3600
df = df.sort_values("date").reset_index(drop=True)

#Calculate recovery scores 
recovery_scores = []
sleep_target = 8.0

for i in range(7, len(df)):
    hrv_baseline = df.loc[i-7:i-1, "avgOvernightHrv"].mean()
    rhr_baseline = df.loc[i-7:i-1, "restingHeartRate"].mean()

    hrv_today = df.loc[i, "avgOvernightHrv"]
    rhr_today = df.loc[i, "restingHeartRate"]
    sleep_today = df.loc[i, "sleep_hours"]

    # lets not divide by 0 or nan
    if pd.isna(hrv_today) or pd.isna(hrv_baseline) or hrv_baseline == 0:
        hrv_score = 0
    else:
        hrv_score = 40 * min(1, hrv_today / hrv_baseline)

    if pd.isna(rhr_today) or pd.isna(rhr_baseline) or rhr_today == 0:
        rhr_score = 0
    else:
        rhr_score = 25 * min(1, rhr_baseline / rhr_today)

    if pd.isna(sleep_today) or sleep_target == 0:
        sleep_score = 0
    else:
        sleep_score = 25 * min(1, sleep_today / sleep_target)
        
    #further integration with other metrics depending on what shows promise in EDA TBC. 
    bonus_score = 10 

    total_score = hrv_score + rhr_score + sleep_score + bonus_score

    recovery_scores.append({
        "date": df.loc[i, "date"],
        "recoveryScore": round(total_score, 1)
    })

recovery_df = pd.DataFrame(recovery_scores)
print(recovery_df)


          date  recoveryScore
0   2024-02-14           59.1
1   2024-02-15           60.0
2   2024-02-16           60.0
3   2024-02-17           47.4
4   2024-02-18           60.0
..         ...            ...
494 2025-06-22           99.1
495 2025-06-23          100.0
496 2025-06-24           95.4
497 2025-06-25           93.5
498 2025-06-26           97.0

[499 rows x 2 columns]


**Update database with the new recovery scores** allows for improvmenets to be made to recovery score in the future

In [41]:
with engine.begin() as conn:
    for _, row in recovery_df.iterrows():
        update_query = text("""
            UPDATE simplesleepdata
            SET "recoveryScore" = :recovery_score
            WHERE date = :date
        """)
        conn.execute(update_query, {
            "recovery_score": row['recoveryScore'],
            "date": row['date'].strftime("%Y-%m-%d")  
        })

        

**Ensure RecoveryScore Update** 

In [43]:
query = text('SELECT date, "recoveryScore" FROM simplesleepdata')
df = pd.read_sql(query, con=engine)
print(df)

           date  recoveryScore
0    2024-02-13            NaN
1    2024-02-12            NaN
2    2024-02-11            NaN
3    2024-02-10            NaN
4    2024-02-09            NaN
..          ...            ...
501  2025-06-22           99.1
502  2025-06-23          100.0
503  2025-06-24           95.4
504  2025-06-25           93.5
505  2025-06-26           97.0

[506 rows x 2 columns]


**Create Sleep Debt/needed Logic**

In [66]:
def estimateSleepNeeded(row, base_sleep):
    # Recovery Score impact
    recovery_score = row.get('recoveryScore')
    if pd.isnull(recovery_score):
        recovery_factor = 0
    elif recovery_score < 50:
        recovery_factor = 1.0
    elif recovery_score < 65:
        recovery_factor = 0.5
    else:
        recovery_factor = 0

    # HRV & RHR impact
    hrv = row.get("avgOvernightHrv")
    rhr = row.get("restingHeartRate")
    hrv_baseline = row.get("hrv_baseline")
    rhr_baseline = row.get("rhr_baseline")

    hrv_rhr_factor = 0
    if pd.notna(hrv) and pd.notna(hrv_baseline) and hrv < 0.9 * hrv_baseline:
        hrv_rhr_factor += 0.5
    if pd.notna(rhr) and pd.notna(rhr_baseline) and rhr > 1.1 * rhr_baseline:
        hrv_rhr_factor += 0.5

    total_factor = recovery_factor + hrv_rhr_factor
    extra_sleep = base_sleep * 0.1 * total_factor


    return base_sleep + extra_sleep

**Updating SleepNeeded Column** /n
This allows for further updates to be able to made ito the logic behind the sleep needed

In [67]:
query = text('SELECT date, "avgOvernightHrv", "restingHeartRate", "sleepDuration", "recoveryScore" FROM simplesleepdata')
df = pd.read_sql(query, con=engine)

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

df["sleep_hours"] = df["sleepDuration"] / 3600
df["hrv_baseline"] = df["avgOvernightHrv"].rolling(7, min_periods=1).mean().shift(1)
df["rhr_baseline"] = df["restingHeartRate"].rolling(7, min_periods=1).mean().shift(1)

base_sleep = df["sleep_hours"].mean()

df["sleepNeeded"] = df.apply(lambda row: estimateSleepNeeded(row, base_sleep), axis=1)


update_query = text("""
    UPDATE simplesleepdata
    SET "sleepNeeded" = :sleep_needed
    WHERE date = :date
""")

with engine.connect() as conn:
    for _, row in df.dropna(subset=['sleepNeeded']).iterrows():
        conn.execute(update_query, {
            "sleep_needed": row['sleepNeeded'],
            "date": row['date'].strftime("%Y-%m-%d")
        })
    conn.commit()



In [68]:
query = text('SELECT date, "sleepNeeded" FROM simplesleepdata')
df = pd.read_sql(query, con=engine)
print(df)

           date  sleepNeeded
0    2024-02-08     7.746763
1    2024-05-26     8.521439
2    2024-02-09     7.746763
3    2024-02-10     7.746763
4    2024-02-11     7.746763
..          ...          ...
501  2025-06-22     7.746763
502  2025-06-23     7.746763
503  2025-06-24     7.746763
504  2025-06-25     7.746763
505  2025-06-26     7.746763

[506 rows x 2 columns]
